<a href="https://colab.research.google.com/github/Sachini-Dissanayaka/DrivenData_PumpItUp/blob/master/ML_PumpItUp_BestScore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Getting Started**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/PumpIt

In [ ]:
!pip install catboost
!pip install category_encoders

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
import datetime as dt
from numpy import mean
from numpy import std

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.impute import KNNImputer

# encoders
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
import category_encoders as ce

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score

# classifiers
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, StackingClassifier, VotingClassifier, GradientBoostingClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
import lightgbm as lgb
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from sklearn import svm

# hyperparameter tuning
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

from sklearn.decomposition import PCA
from matplotlib.pyplot import figure

#**EDA**

#**Data Collection**

In [104]:
features_df = pd.read_csv(
    "TrainingSetValues.csv", 
    index_col="id",
    low_memory=False
)
labels_df = pd.read_csv(
    "TrainingSetLabels.csv", 
    index_col="id"
)
test_df = pd.read_csv(
    "TestSetValues.csv", 
    index_col="id",
    low_memory=False
)

In [105]:
features_df.head()

,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [106]:
X = features_df.copy()
y = labels_df.copy()
X_test = test_df.copy()

In [ ]:
X.info()

In [108]:
# Combine train data with test data
df=X.append(X_test)

In [ ]:
df.describe()

In [ ]:
y.head()

#**Data Cleaning**

In [113]:
X_null = X.isnull().sum()
test_null = X_test.isnull().sum()

In [ ]:
list_features = X_test.columns
print('Number of features : ',len(list_features))
print('')
i = 0
for f in list_features:
    print('feature:', f, '|| Type:', type(X[f][0]), '|| Example:', X[f][0], '|| number of unique values', len(X[f].unique()), '|| missing values training : ', (X_null.values[i]/X.shape[0])*100, '|| missing values test : ', (test_null.values[i]/X_test.shape[0])*100 )
    i +=1

In [ ]:
#finding Zero Percentage in each column
all = X.shape[0]
X_zero = X.astype(bool).sum(axis=0)
list_X = X_test.columns
print('Number of features : ',len(list_X))
print('')
i = 0
for f in list_X:
  if (((all - X_zero.values[i])/all)*100 > 0):
    print('feature:', f, '|| Type:', type(X[f][0]),'|| Example:', X[f][0], '|| zero percentage', ((all - X_zero.values[i])/all)*100 )
  i += 1

In [116]:
#remove columns with zeros and more missing values along with more unique values
df.drop(columns=['num_private', 'scheme_name'], axis=1, inplace=True)

In [117]:
#drop unwanted columns
df.drop(columns=['amount_tsh','recorded_by','wpt_name'], axis=1, inplace=True)

**Handling Missing Values**

In [ ]:
# Number of missing values in each column
missing_val_count_by_column = (df.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

*   num_private - droped - only one unique value
*   wpt_name - droped - number of unique values 37400
*   num_private, scheme_name - droped
*   Population - need to search more 36% zero count - relaced zeros with nan and mean
*   funder - mode
*   installer - mode
*   subvilage - mode
*   public_meeting - mode
*   scheme_management - mode
*   permit - mode
*   construction year - replace zeros with nan and use mode









In [119]:
mis_columns = ['construction_year','population']
for col in mis_columns:
  df[col].replace(0, np.nan, inplace=True)

In [121]:
# Fill missing values with mode
def fill_mode(all_column,X):
  for X_column in all_column:
    X[X_column] = X[X_column].fillna(X[X_column].mode()[0])
  return X

In [122]:
df = fill_mode(["funder","installer","subvillage","public_meeting","scheme_management","permit","construction_year"],df)

In [123]:
# Fill missing values with mean
def fill_mean(all_column,X):
  for X_column in all_column:
    X[X_column] = X[X_column].fillna(round(X[X_column].mean()))
  return X

In [124]:
df = fill_mean(["population"],df)

In [125]:
# Number of missing values in each column
missing_val_count_by_column = (df.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

Series([], dtype: int64)


*   *No missing values*

#**Data Preprocessing**

In [173]:
#date time
df.date_recorded = pd.to_datetime(df.date_recorded)

In [174]:
df['year'] = df['date_recorded'].dt.year
df['month'] = df['date_recorded'].dt.month
df['day_of_months'] = df['date_recorded'].dt.day

In [175]:
df.drop(columns=['date_recorded'], axis=1, inplace=True)

In [176]:
#Payment
df["Payment_all"] = df["payment"] + df["payment_type"]

#quality
df["Quality_all"] = df["water_quality"] + df["quality_group"]

#Quantitiy
df["Quantity_all"] = df["quantity"] + df["quantity_group"]

#waterpoint_type
df["Waterpoint_all"] = df["waterpoint_type"] + df["waterpoint_type_group"]

In [177]:
#drop unwanted columns
df.drop(columns=['payment','payment_type','water_quality','quality_group','quantity','quantity_group','waterpoint_type','waterpoint_type_group'], axis=1, inplace=True)

**Encoding**

In [178]:
#one hot encoding
def one_hot_encode(df,object_cols):
  OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
  OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(df[object_cols]))

  # One-hot encoding removed index; put it back
  OH_cols_train.index = df.index

  # Remove categorical columns (will replace with one-hot encoding)
  num_X_train = df.drop(object_cols, axis=1)

  # Add one-hot encoded columns to numerical features
  df = pd.concat([num_X_train, OH_cols_train], axis=1)
  return df

In [179]:
columns=['source','source_type','source_class','public_meeting','permit','extraction_type','extraction_type_group','extraction_type_class','management','management_group',"basin", "scheme_management","Payment_all","Quality_all","Quantity_all","Waterpoint_all"]
df = one_hot_encode(df, columns)
# one_hot_encode(test, columns)

In [ ]:
#binary encoder
#use if count of unique values is more than 15
 
enc = ce.BinaryEncoder(cols=['funder','installer', 'subvillage', 'region','lga','ward']).fit(df)
df = enc.transform(df)

#**Separate training and test set**

In [188]:
X = df[:len(features_df)]
X_test = df[len(features_df):]

In [ ]:
X.head()

In [ ]:
X_test.head()

In [191]:
X.shape

(59400, 212)

In [192]:
X_test.shape

(14850, 212)

#**Model Training**

In [193]:
def score_dataset(X, y, model):
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=0)
    # Metric is Accuracy
    score = cross_val_score(
        model, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
    print('Accuracy: %.3f (%.3f)' % (mean(score), std(score)))

In [194]:
modelRF = RandomForestClassifier(n_estimators=400, random_state=0,max_depth=20)
modelETC = ExtraTreesClassifier(n_estimators=1000,min_samples_split=10)
modelXGB = XGBClassifier()
modelCAT = CatBoostClassifier(max_ctr_complexity=5, task_type='CPU', iterations=1000,
                   eval_metric='AUC', od_type='Iter', od_wait=500, loss_function = 'MultiClass', random_seed=1, depth=10, learning_rate = 0.03, l2_leaf_reg = 1)
modelMLP = MLPClassifier()
modelSVM = SVC(kernel='linear', C=1,gamma='auto')

In [ ]:
# score model
scoreSVM = score_dataset(X,y,modelSVM)

In [ ]:
# score model
scoreRFTest = score_dataset(X,y,modelRF)

Accuracy: 0.816 (0.004)


In [ ]:
# score model
scoreXGB = score_dataset(X,y,modelXGB)

Accuracy: 0.745 (0.004)


In [ ]:
# score model
scoreETC = score_dataset(X,y,modelETC)

#**Hyperparameter Tunning**

In [ ]:
def grid_search(model, params, X_tune, y_tune):
  clf =  GridSearchCV(model, params, cv=5, scoring='accuracy')
  clf.fit(X_tune, y_tune)
  return clf.best_score_, clf.best_params_

In [ ]:
model_params = {
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [50,100,200,400],
            'max_depth': [5, 10, 20, 100]
        }
    },
    'xg_boost': {
        'model': XGBClassifier(),
        'params' : {
            'eta': [0.01, 0.1, 0.2, 0.4],
            'max_depth': [5, 10, 15, 20, 22, 100]
        }
    },
    'cat_boost': {
        'model': CatBoostClassifier(loss_function = 'MultiClass', iterations=1000),
        'params' : {
            'learning_rate': [0.03, 0.1],
            'depth':[4, 6, 10],
            'l2_leaf_reg': [1, 3, 5, 7, 9]
        }
    }
}

In [ ]:
X_tune=X
y_tune = y.status_group

In [ ]:
best_score, best_params = grid_search(model_params['random_forest']['model'], model_params['random_forest']['params'], X_tune, y_tune)
print(best_score, best_params)

0.8132491582491583 {'max_depth': 20, 'n_estimators': 400}


In [ ]:
best_score, best_params = grid_search(model_params['xg_boost']['model'], model_params['xg_boost']['params'], X_tune, y_tune)
print(best_score, best_params)

In [ ]:
best_score, best_params = grid_search(model_params['cat_boost']['model'], model_params['cat_boost']['params'], X_tune, y_tune)
print(best_score, best_params)

#**Ensembling**

In [195]:
estimators_stacked = [
              #('xbg', modelXGB),
              # ('mlp', modelMLP),
              #('cat',modelCAT),
              ('xt', modelETC),
              ('rn', modelRF)
             ]

# Stacking Classifier
stacked_estimator =  StackingClassifier(estimators=estimators_stacked,
                                    final_estimator=LogisticRegression()  
                                  )
# Voting Classifier
voting_estimator = VotingClassifier(estimators=estimators_stacked, 
                        voting='hard')  # hard if labels needed, soft if probabilites needed

In [ ]:
# score model
scoreEns = score_dataset(X,y,stacked_estimator)

#**Train on entire dataset**

In [ ]:
model = modelRF
X_train = X
y_train = y

model.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=20, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=400,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [97]:
#ETC,RF
modelEns = stacked_estimator
X_train = X
y_train = y

modelEns.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


StackingClassifier(cv=None,
                   estimators=[('xt',
                                ExtraTreesClassifier(bootstrap=False,
                                                     ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='gini',
                                                     max_depth=None,
                                                     max_features='auto',
                                                     max_leaf_nodes=None,
                                                     max_samples=None,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=1,
                                                     min_samples_split=10,
                                                     min_we

In [ ]:
#ETC,RF,cat
modelEns = stacked_estimator
X_train = X
y_train = y

modelEns.fit(X_train, y_train)

#**Predict**

In [ ]:
predict=model.predict(X_test)

In [197]:
predict=modelEns.predict(X_test)

#**Submission**

In [198]:
submission_df = pd.read_csv("SubmissionFormat.csv", 
                            index_col="id")

In [199]:
# Make sure we have the rows in the same order
np.testing.assert_array_equal(test_df.index.values, 
                              submission_df.index.values)

# Save predictions to submission data frame
submission_df["status_group"] = predict

In [200]:
submission_df.to_csv('my_submission.csv', index=True)

In [201]:
from google.colab import files
files.download('my_submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>